# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking the current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
#         next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
#print(len(full_data_rows_list))
# shows what the list of event data rows will look like
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion

## Work with the CSV file titled <font color=red>event_datafile_new.csv</font>.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [3]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [4]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydatabase 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
try:
    session.set_keyspace('sparkifydatabase')
except Exception as e:
    print(e)

### Create tables to run the following queries. Apache Cassandra database modeling is bases on queries you want to run.

## Questions examples to be answered (comes from analyst team)

### 1. Give me the artist, song title and song's length in the music app history that was heard during \ sessionId = 338, and itemInSession = 4

**Table structure**

The query above requires five informations, demanded are two of them: ```session_id``` and ```item_in_session```, as these are identificators. Three others are ```artist```, ```song```, ```song_length```.
These are specifications necessary to build a table with ```session_id```, furthermore it will be clustered by ```item_in_session```. This combination will allow for uniquely identifying each row.

In [6]:
tableQuery1 = "CREATE TABLE IF NOT EXISTS songDetails"
tableQuery1 = tableQuery1 + (""" (session_id INT, 
                        item_in_session INT, 
                        artist TEXT, 
                        song_title TEXT,
                        song_length FLOAT, 
                        PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(tableQuery1)
except Exception as e:
    print(e)

**Data insertion**

In [7]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songDetails(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verification of the SELECT statement

Query is defined as follows

```
SELECT 
    session_id, 
    item_in_session, 
    artist, 
    song, 
    song_length 
FROM 
    session_library 
WHERE 
    session_id = 338 AND
    item_in_session  = 4 
```

In [8]:
query1 = """SELECT artist, song_title, song_length 
            FROM songDetails
            WHERE session_id = %s
                AND item_in_session = %s;
         """
try:
    rows = session.execute(query1, (338,4))
except Exception as e:
    print(e)
    
for row in rows:
    print(f"Artist: {row.artist} \nSong Title: {row.song_title} \nSong Length: {str(row.song_length)}")

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Table structure**
Based on the requirements two identificators will be taken: ```user_id``` and ```session_id```. Field defined: ```artist```, ```song```, ```first_name```, ```last_name```.
It is also requested the results to be ordered by ```item_in_session```. In order for the query to work, this field has to be added to PRIMARY KEY.

In [9]:
tableQuery2 = "CREATE TABLE IF NOT EXISTS artistDetails"
tableQuery2 = tableQuery2 + (""" (user_id INT, 
                                    session_id INT, 
                                    item_in_session INT, 
                                    artist_name TEXT, 
                                    song_title TEXT, 
                                    user_first_name TEXT, 
                                    user_last_name TEXT, 
                                    PRIMARY KEY ((user_id, 
                                                 session_id), 
                                                 item_in_session)
                )""")
try:
    session.execute(tableQuery2)
except Exception as e:
    print(e)

**Data insertion**

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO artistDetails(user_id, 
                                            session_id, 
                                            item_in_session, 
                                            artist_name, 
                                            song_title, 
                                            user_first_name,  
                                            user_last_name
                                            )"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))               

**Verification of the SELECT statement**

Query is defined as follows

```
SELECT 
    artist, 
    song, 
    first_name, 
    last_name, 
    item_in_session 
FROM 
    user_library 
WHERE 
    user_id = 10 AND 
    session_id = 182
```

In [11]:
query2 = """SELECT artist_name, song_title, user_first_name, user_last_name 
                    FROM artistDetails
                    WHERE user_id = %s
                    AND session_id = %s; 
                    """      
try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print(f"Artist: {row.artist_name} \nSong Title: {row.song_title} \nUser First Name: {row.user_first_name} \nUser Last Name: {row.user_last_name}")
    print("------------------")

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
User First Name: Sylvie 
User Last Name: Cruz
------------------
Artist: Three Drives 
Song Title: Greece 2000 
User First Name: Sylvie 
User Last Name: Cruz
------------------
Artist: Sebastien Tellier 
Song Title: Kilometer 
User First Name: Sylvie 
User Last Name: Cruz
------------------
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name: Sylvie 
User Last Name: Cruz
------------------


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

**Table structure**

As the question states, only one key is demanded, that is ```song```. However, in order for query to work properly, i.e., get user's first and last name, ```user_id``` has to be included into PRIMARY KEY.

In [12]:
tableQuery3 = "CREATE TABLE IF NOT EXISTS userDetails"
tableQuery3 = tableQuery3 + (""" (song_title TEXT, 
                                    user_id INT, 
                                    user_first_name TEXT, 
                                    user_last_name TEXT, 
                                    PRIMARY KEY (song_title, user_id)
                )""")
try:
    session.execute(tableQuery3)
except Exception as e:
    print(e)

**Data insertion**

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO userDetails(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

**Verification of the SELECT statement**

Query is defined as follows

```
SELECT 
    first_name, 
    last_name, 
    song, 
    artist 
FROM 
    user_song_library 
WHERE 
    song = 'All Hands Against His Own'
```

In [14]:
query3 = """SELECT user_first_name, user_last_name 
            FROM userDetails 
            WHERE song_title = %s;"""
try:
    rows = session.execute(query3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

for row in rows:
    print(f"User First / Last Name: {row.user_first_name} / {row.user_last_name}")

User First / Last Name: Jacqueline / Lynch
User First / Last Name: Tegan / Levine
User First / Last Name: Sara / Johnson


### Drop the tables before closing out the sessions

In [15]:
session.execute("DROP TABLE IF EXISTS songDetails")
session.execute("DROP TABLE IF EXISTS artistDetails")
session.execute("DROP TABLE IF EXISTS userDetails")

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()